In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input/ubiquantfinalsubmission1'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

../input/ubiquantfinalsubmission1/two128-bestenco-test-1_0.h5
../input/ubiquantfinalsubmission1/two256drop0.2-bestenco-test-1_0.h5
../input/ubiquantfinalsubmission1/del-355-420-best-enco-kfold-test_1.h5
../input/ubiquantfinalsubmission1/512_256512-bestenco-kfold_1.h5
../input/ubiquantfinalsubmission1/encoder-best1-512_128-seed34_m1.h5
../input/ubiquantfinalsubmission1/512_256512-bestenco-kfold_0.h5
../input/ubiquantfinalsubmission1/512_128-512best1-enco_1.h5
../input/ubiquantfinalsubmission1/512_128-512best1-enco_0.h5
../input/ubiquantfinalsubmission1/encoder-best1-512_128-seed34_m2.h5
../input/ubiquantfinalsubmission1/two256drop0.2-bestenco-test-1_2.h5
../input/ubiquantfinalsubmission1/del-355-420-best-enco-kfold-test_0.h5
../input/ubiquantfinalsubmission1/encoder-best1-384-m2.h5
../input/ubiquantfinalsubmission1/two256drop0.2-bestenco-test-1_1.h5
../input/ubiquantfinalsubmission1/encoder-best1-512_128-0.153-model_1.h5
../input/ubiquantfinalsubmission1/encoder-best1-512_128-0.153-mode

In [2]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import activations
from tensorflow.keras import metrics
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
import gc


In [3]:
data = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')
data = data.drop(columns=['time_id', 'row_id', 'investment_id', 'target']).values
scaler = StandardScaler()
scaler.fit(data)
del data
gc.collect()

0

In [4]:
class Mish(layers.Layer):
    def __init__(self, **kwargs):
        super(Mish, self).__init__(**kwargs)

    def call(self, inputs, *args, **kwargs):
        return inputs * K.tanh(K.softplus(inputs))

In [5]:
input_dim = 300

def build_encoder_1(input_dim, noise_rate=0.05):
    ori_input = Input(shape=(input_dim,), dtype='float32')
    encode = layers.BatchNormalization()(ori_input)
    encode = layers.GaussianNoise(stddev=noise_rate)(encode)
    encode = layers.Dense(256, activation=activations.swish)(encode)
    decode = layers.Dropout(0.2)(encode)
    decode = layers.Dense(input_dim, name='decoded')(decode)
    x = layers.Dense(160)(decode)
    x = layers.Activation(activations.swish)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(1, name='target_out')(x)

    encoder = models.Model(inputs=[ori_input], outputs=[encode])
    autoencoder = models.Model(inputs=[ori_input], outputs=[decode, x])
    
    return autoencoder, encoder

autoencoder, encoder_1 = build_encoder_1(input_dim)
encoder_1.trainable = False

2022-04-12 08:40:28.780903: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-12 08:40:28.896579: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-12 08:40:28.897396: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-12 08:40:28.898945: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [6]:
def build_encoder(input_dim, noise_rate=0.05):
    ori_input = Input(shape=(input_dim,), dtype='float32')
    encode = layers.BatchNormalization()(ori_input)
    encode = layers.GaussianNoise(stddev=noise_rate)(encode)
    encode = layers.Dense(256)(encode)
    encode = Mish()(encode)
    decode = layers.Dropout(0.2)(encode)
    decode = layers.Dense(input_dim, name='decoded')(decode)
    x = layers.Dense(265, use_bias=False)(decode)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(1, name='target_out')(x)

    encoder = models.Model(inputs=[ori_input], outputs=[encode])
    autoencoder = models.Model(inputs=[ori_input], outputs=[decode, x])
    return autoencoder, encoder

autoencoder, encoder_0 = build_encoder(input_dim)
encoder_0.trainable = False

In [7]:
def build_model_512_128(encoder):
    ox_input = Input(shape=(input_dim,), dtype='float32')
    ox_enco = encoder(ox_input)

    x = layers.Dense(512, use_bias=False)(ox_input)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activations.swish)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation=activations.swish)(x)
    
    x = layers.Concatenate()([x,  ox_enco])
    x = layers.BatchNormalization()(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activations.swish)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activations.swish)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activations.swish)(x)
    x = layers.Dropout(0.2)(x)
    output = layers.Dense(1)(x)
    model = models.Model(inputs=[ox_input], outputs=[output])

    return model

In [8]:
def build_model_512_128_mish(encoder):
    ox_input = Input(shape=(input_dim,), dtype='float32')
    ox_enco = encoder(ox_input)

    x = layers.Dense(512, use_bias=False)(ox_input)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128)(x)
    x = Mish()(x)

    x = layers.Concatenate()([x, ox_enco])
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.2)(x)
    output = layers.Dense(1)(x)
    model = models.Model(inputs=[ox_input], outputs=[output])
    return model

In [9]:
def build_model_384(encoder):
    ox_input = Input(shape=(input_dim,), dtype='float32')
    ox_enco = encoder(ox_input)

    x = layers.Dense(384, use_bias=False)(ox_input)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activations.swish)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation=activations.swish)(x)
    
    x = layers.Concatenate()([x,  ox_enco])
    x = layers.BatchNormalization()(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activations.swish)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activations.swish)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activations.swish)(x)
    x = layers.Dropout(0.2)(x)
    output = layers.Dense(1)(x)
    model = models.Model(inputs=[ox_input], outputs=[output])

    return model

In [10]:
def build_model_512_256_512(encoder):
    ox_input = Input(shape=(input_dim,), dtype='float32')
    ox_enco = encoder(ox_input)
    #     ox_enco = layers.BatchNormalization()(ox_enco)
    #     ox_enco = layers.Dense(256, activation=activations.swish)(ox_enco)

    #     inv_input = Input(shape=(1,), dtype='int64')
    #     inv_emb = layers.Embedding(inv_vocb, 16)
    #     squeeze_layer = Squeeze()
    #     inv_enco = inv_emb(inv_input)
    #     inv_enco = squeeze_layer(inv_enco)
    x = layers.Dense(512, use_bias=False)(ox_input)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activations.swish)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, activation=activations.swish)(x)

    #     x = layers.BatchNormalization()(x)
    #     x = layers.Activation(activations.swish)(x)

    x = layers.Concatenate()([x, ox_enco])
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activations.swish)(x)
    x = layers.Dropout(0.3)(x)
    #     x = layers.Dropout(0.25)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activations.swish)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activations.swish)(x)
    x = layers.Dropout(0.3)(x)
    #     x = layers.Dense(128, use_bias=False)(x)
    #     x = layers.BatchNormalization()(x)
    #     x = layers.Activation(activations.swish)(x)
    x = layers.Dense(128, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activations.swish)(x)
    x = layers.Dropout(0.2)(x)
    #     x = layers.Dropout(0.1)(x)
    output = layers.Dense(1)(x)
    model = models.Model(inputs=[ox_input], outputs=[output])

    return model

In [11]:
def build_model_512_128_512(encoder):
    ox_input = Input(shape=(input_dim,), dtype='float32')
    ox_enco = encoder(ox_input)
    #     ox_enco = layers.BatchNormalization()(ox_enco)
    #     ox_enco = layers.Dense(256, activation=activations.swish)(ox_enco)

    #     inv_input = Input(shape=(1,), dtype='int64')
    #     inv_emb = layers.Embedding(inv_vocb, 16)
    #     squeeze_layer = Squeeze()
    #     inv_enco = inv_emb(inv_input)
    #     inv_enco = squeeze_layer(inv_enco)
    x = layers.Dense(512, use_bias=False)(ox_input)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activations.swish)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation=activations.swish)(x)
    #     x = layers.BatchNormalization()(x)
    #     x = layers.Activation(activations.swish)(x)

    x = layers.Concatenate()([x, ox_enco])
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activations.swish)(x)
    x = layers.Dropout(0.3)(x)
    #     x = layers.Dropout(0.25)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activations.swish)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activations.swish)(x)
    x = layers.Dropout(0.3)(x)
    #     x = layers.Dense(128, use_bias=False)(x)
    #     x = layers.BatchNormalization()(x)
    #     x = layers.Activation(activations.swish)(x)
    x = layers.Dense(128, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activations.swish)(x)
    x = layers.Dropout(0.2)(x)
    #     x = layers.Dropout(0.1)(x)
    output = layers.Dense(1)(x)
    model = models.Model(inputs=[ox_input], outputs=[output])

    return model

In [12]:
def build_model_two256drop02(encoder):
    ox_input = Input(shape=(input_dim,), dtype='float32')
    ox_enco = encoder(ox_input)

    x = layers.Dense(512, use_bias=False)(ox_input)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128)(x)
    x = Mish()(x)

    x = layers.Concatenate()([x, ox_enco])
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(128, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.2)(x)
    output = layers.Dense(1)(x)
    model = models.Model(inputs=[ox_input], outputs=[output])
    return model

In [13]:
def build_model_dropmove(encoder):
    ox_input = Input(shape=(input_dim,), dtype='float32')
    ox_enco = encoder(ox_input)

    x = layers.Dense(512, use_bias=False)(ox_input)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128)(x)
    x = Mish()(x)

    x = layers.Concatenate()([x, ox_enco])
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dense(128, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.2)(x)
    output = layers.Dense(1)(x)
    model = models.Model(inputs=[ox_input], outputs=[output])
    return model

In [14]:
def build_model_two128(encoder):
    ox_input = Input(shape=(input_dim,), dtype='float32')
    ox_enco = encoder(ox_input)

    x = layers.Dense(512, use_bias=False)(ox_input)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128)(x)
    x = Mish()(x)

    x = layers.Concatenate()([x, ox_enco])
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dense(128, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.2)(x)
    output = layers.Dense(1)(x)
    model = models.Model(inputs=[ox_input], outputs=[output])
    return model

In [15]:


def get_conv1d_model():
    features_inputs = tf.keras.Input((input_dim, ), dtype=tf.float32)
    
    ## feature ##
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dropout(0.1)(feature_x)
    ## convolution 1 ##
    feature_x = layers.Reshape((-1,1))(feature_x)
    feature_x = layers.Conv1D(filters=16, kernel_size=4, strides=1, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)
    ## convolution 2 ##
    feature_x = layers.Conv1D(filters=16, kernel_size=4, strides=4, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)
    ## convolution 3 ##
    feature_x = layers.Conv1D(filters=64, kernel_size=4, strides=1, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)
    ## convolution 4 ##
    feature_x = layers.Conv1D(filters=64, kernel_size=4, strides=4, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)
    ## convolution 5 ##
    feature_x = layers.Conv1D(filters=64, kernel_size=4, strides=2, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)
    ## flatten ##
    feature_x = layers.Flatten()(feature_x)
    
    x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(feature_x)
    
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.1)(x)
    output = layers.Dense(1)(x)
#     rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[features_inputs], outputs=[output])
#     model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse, correlation])
    return model

In [16]:


def get_conv1d_model_0():
    features_inputs = tf.keras.Input((input_dim, ), dtype=tf.float32)
    
    ## feature ##
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dropout(0.1)(feature_x)
    ## convolution 1 ##
    feature_x = layers.Reshape((-1,1))(feature_x)
    feature_x = layers.Conv1D(filters=16, kernel_size=4, strides=1, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = Mish()(feature_x)
    ## convolution 2 ##
    feature_x = layers.Conv1D(filters=16, kernel_size=4, strides=4, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = Mish()(feature_x)
    ## convolution 3 ##
    feature_x = layers.Conv1D(filters=64, kernel_size=4, strides=1, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x =Mish()(feature_x)
    ## convolution 4 ##
    feature_x = layers.Conv1D(filters=64, kernel_size=4, strides=4, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = Mish()(feature_x)
    ## convolution 5 ##
    feature_x = layers.Conv1D(filters=64, kernel_size=4, strides=2, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = Mish()(feature_x)
    ## flatten ##
    feature_x = layers.Flatten()(feature_x)
    
    x = layers.Dense(512, kernel_regularizer="l2")(feature_x)
    x = Mish()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(128, kernel_regularizer="l2")(x)
    x = Mish()(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(32, kernel_regularizer="l2")(x)
    x = Mish()(x)
    x = layers.Dropout(0.1)(x)
    output = layers.Dense(1)(x)
#     rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[features_inputs], outputs=[output])
#     model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse, correlation])
    return model

In [17]:
# 0.1537
model_512_128_m0 = build_model_512_128(encoder_1)
model_512_128_m0.load_weights('../input/ubiquantfinalsubmission1/encoder-best1-512_128-0.153-model_0.h5')
model_512_128_m1 = build_model_512_128(encoder_1)
model_512_128_m1.load_weights('../input/ubiquantfinalsubmission1/encoder-best1-512_128-0.153-model_1.h5')

# 0.1536
model_384_m1 = build_model_384(encoder_1)
model_384_m1.load_weights('../input/ubiquantfinalsubmission1/encoder-best1-384-m1.h5')
model_384_m2 = build_model_384(encoder_1)
model_384_m2.load_weights('../input/ubiquantfinalsubmission1/encoder-best1-384-m2.h5')


# 0.1537
model_512_256_512_m0 = build_model_512_256_512(encoder_1)
model_512_256_512_m0.load_weights('../input/ubiquantfinalsubmission1/512_256512-bestenco-kfold_0.h5')
model_512_256_512_m1 = build_model_512_256_512(encoder_1)
model_512_256_512_m1.load_weights('../input/ubiquantfinalsubmission1/512_256512-bestenco-kfold_1.h5')
model_512_256_512_m2 = build_model_512_256_512(encoder_1)
model_512_256_512_m2.load_weights('../input/ubiquantfinalsubmission1/512_256512-bestenco-kfold_2.h5')

# 0.1533
model_512_128_512_m0 = build_model_512_128_512(encoder_1)
model_512_128_512_m0.load_weights('../input/ubiquantfinalsubmission1/512_128-512best1-enco_0.h5')
model_512_128_512_m1 = build_model_512_128_512(encoder_1)
model_512_128_512_m1.load_weights('../input/ubiquantfinalsubmission1/512_128-512best1-enco_1.h5')
model_512_128_512_m2 = build_model_512_128_512(encoder_1)
model_512_128_512_m2.load_weights('../input/ubiquantfinalsubmission1/512_128-512best1-enco_2.h5')


# 0.1538, mish-best-enco
model_two256drop_m0 = build_model_two256drop02(encoder_1)
model_two256drop_m0.load_weights('../input/ubiquantfinalsubmission1/two256drop0.2-bestenco-test-1_0.h5')
model_two256drop_m1 = build_model_two256drop02(encoder_1)
model_two256drop_m1.load_weights('../input/ubiquantfinalsubmission1/two256drop0.2-bestenco-test-1_1.h5')
model_two256drop_m2 = build_model_two256drop02(encoder_1)
model_two256drop_m2.load_weights('../input/ubiquantfinalsubmission1/two256drop0.2-bestenco-test-1_2.h5')

# 0.1534, two1280mish
model_two128_m0 = build_model_two128(encoder_0)
model_two128_m0.load_weights('../input/ubiquantfinalsubmission1/two128-bestenco-test-1_0.h5')
model_two128_m1 = build_model_two128(encoder_0)
model_two128_m1.load_weights('../input/ubiquantfinalsubmission1/two128-bestenco-test-1_1.h5')
model_two128_m2 = build_model_two128(encoder_0)
model_two128_m2.load_weights('../input/ubiquantfinalsubmission1/two128-bestenco-test-1_2.h5')

# 0.1543
model_dropmove_m0 = build_model_dropmove(encoder_1)
model_dropmove_m0.load_weights('../input/ubiquantfinalsubmission1/256dropmove-bestenco-test-1_0.h5')
model_dropmove_m1 = build_model_dropmove(encoder_1)
model_dropmove_m1.load_weights('../input/ubiquantfinalsubmission1/256dropmove-bestenco-test-1_1.h5')
model_dropmove_m2 = build_model_dropmove(encoder_1)
model_dropmove_m2.load_weights('../input/ubiquantfinalsubmission1/256dropmove-bestenco-test-1_2.h5')

# 0.152
model_512_128_seed34_m0 = build_model_512_128(encoder_1)
model_512_128_seed34_m0.load_weights('../input/ubiquantfinalsubmission1/encoder-best1-512_128-seed34_m0.h5')
model_512_128_seed34_m1 = build_model_512_128(encoder_1)
model_512_128_seed34_m1.load_weights('../input/ubiquantfinalsubmission1/encoder-best1-512_128-seed34_m1.h5')
model_512_128_seed34_m2 = build_model_512_128(encoder_1)
model_512_128_seed34_m2.load_weights('../input/ubiquantfinalsubmission1/encoder-best1-512_128-seed34_m2.h5')

# 全数据model
# 0.1535
autoencoder, encoder_2 = build_encoder(input_dim)
encoder_2.trainable = False
autoencoder, encoder_3 = build_encoder(input_dim)
encoder_3.trainable = False
autoencoder, encoder_4 = build_encoder(input_dim)
encoder_4.trainable = False
model_0 = build_model_two256drop02(encoder_2)
model_0.load_weights('../input/ubiquantfulltrainsubweights/m0-fulltr-two-256-drop0.2-encodrop0.2-ep7-loss0.8171.h5')
model_1 = build_model_dropmove(encoder_3)
model_1.load_weights('../input/ubiquantfulltrainsubweights/m1-fulltr-256dropmove-encodrop0.2-ep7-loss0.8178.h5')
model_2 = build_model_512_128_mish(encoder_4)
model_2.load_weights('../input/ubiquantfulltrainsubweights/m2-fulltr-Mish-512_128-512-newenco-ep7-loss0.8178.h5')

# conv1d,0.1532
conv_models = []
for i in range(5):
    conv_model = get_conv1d_model()
    conv_model.load_weights(f'../input/umpconv1d153/model_{i}.tf')
    conv_models.append(conv_model)
print(len(conv_models))

# 1538
conv_models_1 = []
for i in range(5):
    conv_model = get_conv1d_model()
    conv_model.load_weights(f'../input/ubiqunatconv1d21538/model_{i}.tf')
    conv_models_1.append(conv_model)
print(len(conv_models_1))

# 1536
conv_models_2 = []
for i in range(5):
    conv_model = get_conv1d_model_0()
    conv_model.load_weights(f'../input/ubiquantconvmish/model_{i}.tf')
    conv_models_2.append(conv_model)
print(len(conv_models_2))

5
5
5


In [18]:
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()  

In [19]:
def make_test_dataset(feature, batch_size=2048):
    ds = tf.data.Dataset.from_tensor_slices(((feature)))
    ds = ds.batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)
    return ds

In [20]:
w = [0.1537, 0.1536, 0.1537, 0.1533, 0.1532, 0.1532, 0.1538, 0.1543, 0.1538, 0.1534, 0.1530, 0.1536]
for (test_df, sample_prediction_df) in iter_test:
    try:
        data_ox = test_df.drop(columns=['row_id', 'investment_id']).values
        test_ds = make_test_dataset(data_ox)

        t5 = conv_models[0].predict(test_ds, batch_size=2048)
        t6 = conv_models_1[0].predict(test_ds, batch_size=2048)
        t11 = conv_models_2[0].predict(test_ds)

        for i in range(1, 5):
            t5 += conv_models[i].predict(test_ds, batch_size=2048)
            t6 += conv_models_1[i].predict(test_ds, batch_size=2048)
            t11 += conv_models_2[i].predict(test_ds)
        t6 = t6 /2.0
        t11 = t11/ 2.0
        
        
        data_ox = scaler.transform(data_ox)
        test_ds = make_test_dataset(data_ox)

        t0 = (model_512_128_m0.predict(test_ds, batch_size=2048) +
              model_512_128_m1.predict(test_ds, batch_size=2048))

        t1 = (model_384_m1.predict(test_ds, batch_size=2048) +
              model_384_m2.predict(test_ds, batch_size=2048))

        t7 = (model_dropmove_m0.predict(test_ds, batch_size=2048) +
              model_dropmove_m1.predict(test_ds, batch_size=2048) +
              model_dropmove_m2.predict(test_ds, batch_size=2048))

        t2 = (model_512_256_512_m0.predict(test_ds, batch_size=2048) +
              model_512_256_512_m1.predict(test_ds, batch_size=2048) +
              model_512_256_512_m2.predict(test_ds, batch_size=2048))

        t3 = (model_512_128_512_m0.predict(test_ds, batch_size=2048) +
             model_512_128_512_m1.predict(test_ds, batch_size=2048) +
             model_512_128_512_m2.predict(test_ds, batch_size=2048))

        t4 = (model_0.predict(test_ds, batch_size=2048) +
             model_1.predict(test_ds, batch_size=2048) +
             model_2.predict(test_ds, batch_size=2048))

        t8 = (model_two256drop_m0.predict(test_ds) +
             model_two256drop_m1.predict(test_ds) +
             model_two256drop_m2.predict(test_ds))

        t9 = (model_two128_m0.predict(test_ds) +
             model_two128_m1.predict(test_ds) +
             model_two128_m2.predict(test_ds))

        t10 = (model_512_128_seed34_m0.predict(test_ds) +
              model_512_128_seed34_m1.predict(test_ds) +
              model_512_128_seed34_m2.predict(test_ds))


        target_list = [t0, t1, t2, t3, t4, t5, t6, t7, t8, t9, t10, t11]
        model_num = [2.0, 2.0, 3.0, 3.0, 3.0, 5.0, 5.0, 3.0, 3.0, 3.0, 3.0, 5.0]
        for i in range(len(target_list)):
            target_list[i] = (target_list[i] * 10 / model_num[i] * w[i])
        
        temp_tar = sum(target_list)
        temp_tar[np.isnan(temp_tar)] = 0
        sample_prediction_df['target'] = temp_tar
        print('OK')
        
    except Exception:
        sample_prediction_df['target'] = np.random.randn(len(sample_prediction_df))

    env.predict(sample_prediction_df)  

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


2022-04-12 08:40:42.409151: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-04-12 08:40:44.129771: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


OK
OK
OK
OK
